## SBST Core Usage

This Python notebook file explains the example usage of sbst_core.py

1. Import requirements

In [773]:
# Import requirements
from sbst_core import instrument_and_load, FitnessCalculator, hill_climbing_search
import ast
import random

2. Load the basic information of functions and their branches

In [774]:
file_path = "../benchmark/ex1.py" # path to target

source = open(file_path).read()   
namespace, traveler, record, instrumented_tree = instrument_and_load(source)

# Example: pick first function
func_info = traveler.functions[0]
func_name = func_info.name
func_args = func_info.args
func_dims = func_info.args_dim
func_obj = namespace[func_name]

print(func_info)

 'foo' test 
 'bar' test 
Function foo with 2 arg(s): ['x', 'y']


3. List the branches in the function and choose branch

In [775]:
# list branches in the function
branches = traveler.branches.get(func_name, {})
print("Branches:", branches.keys())

# Choose a branch lineno and an outcome to target (True: taken, False: not taken)
target_lineno = next(iter(branches.keys())) # e.g., Choose the first branch
branch_info = branches[3]
target_branch_node = branch_info.node
target_outcome = True   # e.g. try to make branch taken
subject_node = branch_info.subject
parent_map = traveler.parent_map

print(branch_info)

Branches: dict_keys([3, 4])
Branch If at lineno=3


4. Create fitness calculator and calculate fitness for initial random input

In [776]:
# create fitness calculator
fitness_calc = FitnessCalculator(traveler, record, namespace)

# make an initial candidate (random ints for each arg)
initial = [random.randint(-20, 20) for _ in func_args]

fitness = fitness_calc.fitness_for_candidate(func_obj, initial, target_branch_node, target_outcome, subject_node, parent_map)
print("initial fitness:", fitness, "for", initial)

initial fitness: 0.06008799579808699 for [-20, -11]


5-1. (Optional step) Get actual variables and start hill climbing

In [777]:
# get actual indices that matter (args appearing in branch)
actual_vars = {n.id for n in ast.walk(target_branch_node) if isinstance(n, ast.Name)}
actual_indices = [i for i, a in enumerate(func_args) if a in actual_vars]
if not actual_indices:
    actual_indices = list(range(len(func_args)))

# hill-climb
result = hill_climbing_search(func_obj, initial, target_branch_node, target_outcome, fitness_calc, actual_indices, parent_map, subject_node)
print("found:", result)

found: [42, -9]


5-2. ...or attach to simple HC

In [ ]:
import itertools
import random

def generate_neighbors(current_inputs: list[int], step_size: int) -> list[list[int]]:
    steps = itertools.product([0, -step_size, step_size], repeat=len(current_inputs))
    neighbors = []

    for step in steps:
        if all(s == 0 for s in step): continue # Skip the inputs themselves
        neighbors.append([input + s for input, s in zip(current_inputs, step)])

    return neighbors

def hill_climb(step: int = 1, max_iter: int = 10000) -> tuple[list[int], float, int]:
    x = [random.randint(-100, 100) for _ in func_args]
    fitness = fitness_calc.fitness_for_candidate(func_obj, x, target_branch_node, target_outcome, subject_node, parent_map)

    for i in range(max_iter):
        neighbors = generate_neighbors(x, step)
        scored = [(n, fitness_calc.fitness_for_candidate(func_obj, n, target_branch_node, target_outcome, subject_node, parent_map)) for n in neighbors]

        best_neighbor, best_neighbor_fitness = min(scored, key=lambda t: t[1])

        if best_neighbor_fitness <= fitness:
            x, fitness = best_neighbor, best_neighbor_fitness

        if fitness == 0:
            break

        if i % 10 == 0:
            print(f'Iteration #{i} best x: {x} (fitness: {fitness})')

    return x, fitness, i

best_x, best_f, iteration_n = hill_climb()
print(f"Found best after {iteration_n} iters. Best:", best_x, "Fitness:", best_f)

Iteration #0 best x: [-78, 67] (fitness: 0.11302638191249648)
Iteration #10 best x: [-68, 67] (fitness: 0.10411662529548615)
Iteration #20 best x: [-58, 67] (fitness: 0.09511736910221391)
Iteration #30 best x: [-48, 67] (fitness: 0.0860277142986774)
Iteration #40 best x: [-38, 67] (fitness: 0.07684675281996967)
Iteration #50 best x: [-28, 67] (fitness: 0.06757356747956256)
Iteration #60 best x: [-18, 67] (fitness: 0.05820723187767918)
Iteration #70 best x: [-8, 67] (fitness: 0.04874681030874539)
Iteration #80 best x: [2, 67] (fitness: 0.03919135766791193)
Iteration #90 best x: [12, 67] (fitness: 0.02953991935663791)
Iteration #100 best x: [22, 67] (fitness: 0.01979153118732535)
Iteration #110 best x: [32, 67] (fitness: 0.009945219286995877)
Found best after 120 iters. Best: [42, 67] Fitness: 0.0
